In [1]:
import requests
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Requests
- http://docs.python-requests.org/en/master/

##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 한다.
- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기
- http://m.stock.naver.com

In [2]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목", "시세", "전일비", "등락율", "시가총액", "거래량"])
    for company in companys:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }
    return df

In [3]:
url = make_url(100,1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
95,금호석유,91100,-8900,-8.90,27756,334156
96,포스코대우,22250,-700,-3.05,27451,565949
97,유한양행,217000,3500,1.64,26494,52686
98,대림산업,76300,-600,-0.78,26552,294497
99,메리츠화재,23900,-350,-1.44,26371,78612


##### Dark Sky API
- 날씨 정보를 알려주는 api
- https://darksky.net/dev
- pip3 install python-forecastio

In [4]:
import forecastio
FORECAST_TOKEN = "b41a8726e6a5f4e91e624f70eb66af47"

In [5]:
def forecast(lat,lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["hourly"]["summary"], json_info["timezone"]

In [6]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

('Partly cloudy throughout the day.', 'Asia/Seoul')

In [7]:
def forecast(lat,lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHour = forecast.hourly()
    return byHour.summary

In [8]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

'Partly cloudy throughout the day.'

### BeautifulSoup
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- pip3 install bs4

##### 네이버 검색어 순위
- bs4 사용 : html element를 selector를 사용한다.
- 네이버 검색어 순위를 가져와 데이터 프레임으로 만들기
- http://naver.com

In [9]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://naver.com")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll .ah_l .ah_item")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df

In [10]:
naver_df = naver_top20()
naver_df

,rank,keyword
0,1,en
1,2,대만 지진
2,3,최영미 괴물
3,4,김국진
4,5,최영미
5,6,내보험찾아줌
6,7,연애담
7,8,조권
8,9,쯔위
9,10,1987


##### 다음 검색어 순위

In [11]:
def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://daum.net")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#mArticle ol.list_hotissue.issue_row.list_mini > li")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text.replace("위",""),
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df

In [12]:
daum_df = daum_top10()
daum_df

,rank,keyword
0,1,en
1,2,대만 지진
2,3,이현주
3,4,김승현
4,5,조권
5,6,최영미
6,7,김국진
7,8,부산시 교육청
8,9,류근
9,10,북한 응원단


##### 중복된 키워드 찾아서 출력하기

In [13]:
result = [keyword for keyword in daum_df["keyword"] if naver_df["keyword"].str.contains(keyword).any() ]
result

['en', '대만 지진', '이현주', '조권', '최영미', '김국진']

### file download

In [14]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    download_path = "./data/" + title
    size = 0
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                size += 1024
                f.write(chunk)
    return size

In [15]:
title = "iPhone X is Here — Apple.mp4"
download_link = "http://bit.ly/2FLpRF9"
size = download(title, download_link)
print("download done : {} Mbyte".format(round(size/1024/1024,2)))

FileNotFoundError: [Errno 2] No such file or directory: './data/iPhone X is Here — Apple.mp4'